# Problem 19
##  Counting sundays
------

You are given the following information, but you may prefer to do some research for yourself.

* 1 Jan 1900 was a Monday.
* Thirty days has September,   
  April, June and November.   
  All the rest have thirty-one,   
  Saving February alone,   
  Which has twenty-eight, rain or shine.   
  And on leap years, twenty-nine.
* A leap year occurs on any year evenly divisible by 4, but not on a century unless it is divisible by 400.

*How many Sundays fell on the first of the month during the twentieth century (1 Jan 1901 to 31 Dec 2000)?*

---
Correct result: **171**

### Discussion

The most straightforward approach to the problem is to simply iterate over the days between the start and end points, checking whether each is a Sunday and then incrementing the count if so. This method would have a running time proportional to the size of the date range being checked.

An improvement could be made by skipping ahead by the number of dates in each month, as the first day of each month is only one in which we are interested. This still has a running time proportional to the length of the range, but reduces the number of iterations by a constant fraction.

Similarly, instead of checking candidate dates, we could start from the first known Sunday that falls on the first of the month and then add month totals until the sum is a multiple of 7 (assuring that the first of the corresponding month would also be a Sunday) and then jump to that date. This is the second approach implemented below. Like skipping from first to first, the running time is still proportional to the size of the date range, but reduced by a constant fraction.

Finally, because the pattern of dates and days of the week is regular, over a long enough time period it would be possible to calculate the total without iterating over the whole range. However, the pattern has a period too long to be used as a whole. A sub pattern (applicable to ranges of times that do not cross over years divisible by 400) could be used, however. Even in a four-year leap year cycle there are $365 \cdot 3 + 366 = 1461$ days, and to find a cycle after which we end up on the same day, we would need to find a multiple of this period which is divisible by 7 (the number of days of the week). $1461\bmod 7 = 5$, so that cycle must be 7 four-year periods long or 28 years.

This would mean that we can calculate the number of Sundays on the first in the first 28 year period, multiply this by 3 to find the total for the first 84 years, and then calculate and add the remaining total (from 1985 to the end of the century). This is only a very small improvement on the second method for the range in question.

In [1]:
def get_month_length(month, year):
    if month in (0, 2, 4, 6, 7, 9, 11):
        return 31
    elif month in (3, 5, 8, 10):
        return 30
    else:
        # February
        if (year % 4 != 0):
            return 28
        elif (year % 400 == 0):
            return 29
        elif (year % 4 == 0) and (year % 100 != 0):
            return 29
        else:
            return 28

        
def count_sundays_day_by_day(end_year=2001):
    count = 0
    day = 0
    day_of_the_week = 1
    month = 0
    year = 1901
    while year < end_year:
        if day_of_the_week == 6 and day == 0:
            count += 1
        day = (day + 1) % get_month_length(month, year)
        day_of_the_week = (day_of_the_week + 1) % 7
        if day == 0:
            month = (month + 1) % 12
            if month == 0:
                year += 1
    return count


def count_sundays_skip_by_month(start_year=1901, end_year=2001):
    if start_year >= end_year or start_year < 1901:
        return -1
    count = 0
    day = 0
    day_of_the_week = 1
    month = 0
    year = start_year
    while year < end_year:
        if day_of_the_week == 6 and day == 0:
            count += 1
        month_length = get_month_length(month, year)
        day = 0
        day_of_the_week = (day_of_the_week + month_length) % 7
        if day == 0:
            month = (month + 1) % 12
            if month == 0:
                year += 1        
    return count


def count_sundays_skip_by_mult7(end_year=2001):
    if end_year < 1902:
        return -1
    count = 1
    month = 8
    year = 1901
    while True:
        mult_7 = False
        days_to_next = 0
        months_between = 0
        while not mult_7:
            days_to_next += get_month_length(month, year)
            months_between += 1
            mult_7 = days_to_next % 7 == 0
            month = (month + 1) % 12
            if month == 0:
                year += 1   
            if year >= end_year:
                return count
        count += 1
        if year > end_year:
            return -1
        
    
def count_sundays_using_periodicity():
    return count_sundays_skip_by_month(1901, 1929) * 3 + count_sundays_skip_by_month(1985, 2001)

In [2]:
# Running and timing the approaches
from utils import computation_timer

results = computation_timer({'name':'Counting day-by-day', 'func': count_sundays_day_by_day},
                            {'name':'Counting skipping by month', 'func': count_sundays_skip_by_month},
                            {'name':'Counting skipping by multiples of 7', 'func': count_sundays_skip_by_mult7},
                            {'name':'Counting using periodicity', 'func': count_sundays_using_periodicity})
print("Timed Results:")
for result in results:
    print("\t%s:" % result['name'])
    print("\t\tResult: %d, obtained in %f seconds" % (result['result'], result['running_time']))

Timed Results:
	Counting day-by-day:
		Result: 171, obtained in 0.011816 seconds
	Counting skipping by month:
		Result: 171, obtained in 0.000449 seconds
	Counting skipping by multiples of 7:
		Result: 171, obtained in 0.000392 seconds
	Counting using periodicity:
		Result: 171, obtained in 0.000172 seconds
